In [3]:
import numpy as np
from Constants import *
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print("Num GPUs Available: " + str(len(tf.config.experimental.list_physical_devices('GPU'))))
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(str(len(gpus)) + " Physical GPUs, " + str(len(logical_gpus)) + " Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2.1.0
Num GPUs Available: 1
1 Physical GPUs, 1 Logical GPUs


In [4]:
#TODO: Make this actually do something
def convert_single_state(information):
    # The transposes are so it is in W, H format
    # it shouldn't matter as it could just learn the transpose
    walls = np.array([[1, 1, 1, 1, 1, 1, 1],
                      [1, 0, 0, 0, 0, 0, 1],
                      [1, 1, 0, 1, 0, 1, 1],
                      [0, 0, 0, 1, 0, 0, 0],
                      [1, 1, 0, 1, 0, 1, 1],
                      [1, 0, 0, 0, 0, 0, 1],
                      [1, 1, 1, 1, 1, 1, 1]]).T
    dots = np.array([[0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 1, 1, 1, 0, 0],
                     [0, 0, 1, 0, 1, 0, 0],
                     [0, 0, 1, 0, 1, 0, 0],
                     [0, 0, 1, 0, 1, 0, 0],
                     [0, 0, 1, 1, 1, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0]]).T
    powerup = np.array([[0, 0, 0, 0, 0, 0, 0],
                        [0, 1, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0],
                        [0, 1, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 0]]).T
    fruit = np.array([[0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0], 
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0]]).T
    pacman = np.array([[0, 0, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0], 
                       [0, 1, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0]]).T
    # Red Ghost
    blinky = np.array([[0, 0, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0], 
                       [1, 0, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0],
                       [0, 0, 0, 0, 0, 0, 0]]).T
    # Pink Ghost
    pinky = np.array([[0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0], 
                      [0, 0, 0, 0, 0, 0, 1],
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0]]).T
    # Blue Ghost
    inky = np.array([[0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0], 
                     [0, 0, 0, 0, 0, 1, 0],
                     [0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0]]).T
    # Orange Ghost
    clyde = np.array([[0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0], 
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0],
                      [0, 1, 0, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0, 0, 0]]).T

    return np.array([walls, dots, powerup, fruit, pacman, blinky, pinky, inky, clyde])

In [5]:
# TODO: Make this actually do something
def convert_state_to_input(all_information):
    result = np.zeros((TIMESTEP_PLANES*INPUT_TIMESTEPS, PLANE_WIDTH, PLANE_HEIGHT))
    for j in range(INPUT_TIMESTEPS):
        i = max(len(all_information) - 1 - j, 0)
        result[(TIMESTEP_PLANES * j):(TIMESTEP_PLANES * (j + 1))] = convert_single_state(all_information[i])
    return result

In [55]:
# https://keras.io/models/model/
def create_model_model_api():
    inputs = keras.layers.Input(shape=(TIMESTEP_PLANES*INPUT_TIMESTEPS, PLANE_WIDTH, PLANE_HEIGHT))
    conv = keras.layers.Conv2D(filters=4, 
                               kernel_size=3, 
                               strides=(1, 1), 
                               data_format="channels_first", 
                               activation="relu")(inputs)
    
    flat = keras.layers.Flatten(data_format="channels_first")(conv)
    
    d1 = keras.layers.Dense(8, activation='relu')(flat)
    d2 = keras.layers.Dense(4, activation='tanh')(d1)
    
    return keras.models.Model(inputs=inputs, outputs=d2)

# https://keras.io/models/sequential/
def create_model_sequential_api():
    return keras.models.Sequential([
        keras.layers.Conv2D(filters=4,
                            kernel_size=3,
                            strides=(1, 1),
                            data_format="channels_first",
                            activation="relu",
                            input_shape=(TIMESTEP_PLANES*INPUT_TIMESTEPS, PLANE_WIDTH, PLANE_HEIGHT)),
        keras.layers.Flatten(data_format="channels_first"),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(4, activation='tanh')
    ])

model = create_model_sequential_api()

# https://keras.io/models/model/#compile
model.compile(
    optimizer=keras.optimizers.SGD(lr=1e-4, decay=0, momentum=0),
    loss=keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

# https://keras.io/models/model/#predict
predictions = model.predict(np.array([convert_state_to_input(["b"])]))
print(tf.nn.softmax(predictions).numpy())

# This doesn't quite work right now (I think because there is only one element in the
# set it is using to train)

# https://keras.io/models/model/#fit
model.fit(
    x=np.array([convert_state_to_input(["a"])]),
    y=np.array([[0, 1, 0, 0]]),
    epochs=10
)

# https://keras.io/models/model/#predict
predictions = model.predict(np.array([convert_state_to_input(["b"])]))
print(tf.nn.softmax(predictions).numpy())


[[0.23089646 0.27773488 0.2054334  0.28593525]]
Train on 1 samples
Epoch 1/10
1/1 [==============================] - 0s 202ms/sample - loss: 3.3034 - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 2ms/sample - loss: 2.6261 - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 2ms/sample - loss: 1.6616 - accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 3ms/sample - loss: 1.3743 - accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 3ms/sample - loss: 1.1484 - accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 3ms/sample - loss: 0.9456 - accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 4ms/sample - loss: 0.7506 - accuracy: 0.0000e+00
Epoch 8/10
1/1 [==============================] - 0s 4ms/sample - loss: 0.5446 - accuracy: 0.0000e+00
Epoch 9/10
1/1 [==============================] - 0s 3ms/sample - loss: 0.3064 - accuracy: 0.0000e+00
Epoch 10/10
1